In [1]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1920,
        'height': 1080,
        'scroll': True,
})

{'width': 1920, 'height': 1080, 'scroll': True}

# Week 03, ASTR 596: Fundamentals of Data Science

### MLE, Uncertainty, and Philosophy

### Gautham Narayan 
##### <gsn@illinois.edu>


##### With contributions totally ripped of from Zjelko Ivezic and Mario Juric (UW),  Federica Bianco (U. Del), Phil Marshall and Adam Mantz (SLAC), Kaisey Mandel (Cambridge)

# <center> Quick review of Homework 1 </center>

## Recap

* Describing random variables
    * null hypothesis rejection testing
    * outlier rejection with $\sigma$-clipping and robust statistics
* M-estimators and maximum likelihood
    * HW includes using Huber loss as a robust M-estimator
* the Fisher matrix to get model uncertainties and the Cramer-Rao bound
* Goodness of fit


# <center> It's fine if the last two items seem vague - we've got all this week on maximum likelihood! </center>

### <center> Other things we should talk more about? </center>

### Review:

With both the method of moments, and robust statistics:
* we asserted the data followed some underlying distribution - a **model** for the data
* and then estimated the parameters of that model with sample moments/ robust L-estimators  
   * this let us do *Null Hypotheses Rejection Testing (NHRT)*
   
This is a critical step for us because:
* theories should be **falsifiable**
* analysis should be **reproducible** 

### Review:

* With M-estimators, you defined a loss function
    * loss functions are functions of the quantity you are trying to estimate from the data
    * these have to be chosen to have "nice" properties 
    * there is no ONE TRUE ANSWER i.e. model choice depends on what you are trying to learn from the data
    * you minimize the loss and the location of the minimum gives you the value of estimator
* Directly related to maximum likelihood estimation (MLE) 
    * minimizing the loss = maximizing the likelihood
    * you've seen this with the sum of squared residuals
    * - Log-likelihood, $\chi^2$ and goodness of fit
* This framework also allows you to get some notion of uncertainty
    * Fisher matrix

<img src="nhrt_key.png"> 
Courtesy Federica Bianco

### But there are some natural questions that arise:

* How do you know you specified the right model if you can't rule it out with the observations?
* How do you know you had enough data to estimate the parameters of the model robustly?

### Big Picture: What Are We After?

There are two fundamental types of statistical questions we want to answer:

#### 1. Model Selection

*Given two potential Models, which better describes my data?*

Examples:

- Is there a linear trend in this data?
- Does a linear or quadratic fit describe our data better?
- Is there a periodic signal in this timeseries?
- Does this star have a planet around it? Does this star have two planets around it?

Often one of the two models is a *null hypothesis*, or a baseline model in which the effect you're interested in is not observed.

#### 2. Model Fitting
*Given this Model, what parameters best fit my data?*

Examples:

- What are the slope and intercept of a line of best-fit?
- What are the parameters of the best quadratic fit?
- What is the frequency, amplitude, and phase of a sinusoidal fit?
- What are the orbital parameters of a planet in this radial velocity data?

Simply, how confident are you in your model itself?

#### This is on the surface a straightforward question, but it leads to a deep philosophical split.

### Frequentism

In this view of the Universe, there is some underlying truth.

I we the photon flux F from a given star, then measure it again, then again, and so on, each time I will get a slightly different answer due to the statistical error of my measuring device.

In the limit of a large number of measurements, the frequency of any given value indicates the probability of measuring that value.

For *frequentists* probabilities are fundamentally related to frequencies of events i.e. **$P(D|H)$**.

This means, for example, that in a strict frequentist view, it is meaningless to talk about the probability of the true flux of the star: the true flux is (by definition) a single fixed value. 

To talk about a frequency distribution for a fixed value or model parameter is nonsense.

<img align="left" src="41114_2018_17_Fig4_HTML.png" width="400px">

### Let's look at an actual example from the literature

Fig 4: These plots illustrate the differences between $\Lambda$CDM and Galileon models (see Sect. 7.3.1), with (**GN: Solid lines**) and without massive neutrinos (**GN: Dashed lines**). The Galileon models have background Friedmann equations that contain a scalar-field energy density contribution that generates late time cosmic acceleration and has an evolution consistent with observations and thus similar to that of a $\Lambda$CDM model.

The Top: CMB temperature power spectra showing the ISW effect at low multipoles. 

Middle: CMB lensing potential spectra. Bottom: linear matter power spectra. The models plotted in dashed lines indicate **their best fit models to Ade et al. (2014c) temperature data, WMAP9 polarization data (Hinshaw et al. 2013), and Planck-2013 CMB lensing (Ade et al. 2014d).** 

https://link.springer.com/article/10.1007/s41114-018-0017-4

# The thing to note here is that there is a model that has been "fit" to some noisy data, but the model is taken as "Truth." 

# There is no uncertainty reported about the model. 

<img src="build/1.png">

<img src="build/2.png">

<img src="build/3.png">

<img src="build/4.png">

<img src="build/5.png">

<img src="build/6.png">

<img src="build/7.png">

### The Bayesian View

For Bayesians, the concept of probability is extended to cover degrees of certainty about statements. 

Say a Bayesian claims to measure the flux F of a star with some probability P(F): that probability can certainly be estimated from frequencies in the limit of a large number of repeated experiments, but this is not fundamental. 

The probability is a statement of my knowledge of what the measurement result will be. 

For Bayesians, probabilities are fundamentally related to our own knowledge about an event. 

This means, for example, that in a Bayesian view, we can meaningfully talk about the probability that the true flux of a star lies in a given range. 

That probability codifies our knowledge of the value based on prior information and/or available data i.e. **$P(H|D)$**

For more a more fleshed-out discussion of these different definitions and their consequences, you can see Jake VanderPlas' [series of blog posts](http://jakevdp.github.io/blog/2014/03/11/frequentism-and-bayesianism-a-practical-intro/) on the topic.

The two quantities, $P(H|D)$ and $P(D|H)$ are related to each other by Bayes' rule:

# $$ P(H|D) \propto P(D|H) \cdot P(H) $$

### Wherefore the Controversy?

When we write Bayes' rule this way, we're all of a sudden doing something controversial: can you see where this controversy lies?

Two controversial points:

- We have a probability distribution over model parameters. A frequentist would say this is meaningless!

- The answer depends on the prior $P(H)$. This is the probability of the model without any data: how are we supposed to know that?

Nevertheless, applying Bayes' rule in this manner gives us a means of quantifying our knowledge of the parameters $\theta$ of some hypothesis $H$ given observed data $D$.

## What is the Point?

At first blush, this might all seem needlessly complicated. Why not simply maximize the likelihood and be done with it? Why multiply by a prior at all?

There are a couple good reasons to go through all of this:

### "Purity"

Many advocates of the Bayesian approach argue for it's philosophical purity: you quantify knowledge in terms of a probability, then follow the math to compute the answer.

The fact that you need to specify a prior might be inconvenient, but we can't simply pretend it away.

There are good reasons to think that the Bayesian posterior is just the quantity we wish to compute; in that case we should compute it, however inconvenient.

Perhaps the most vocal 20th century proponent of this view was Jaynes; I'd highly suggest looking at his book, *Probability Theory: The Logic of Science* ([PDF here](http://bayes.wustl.edu/etj/prob/book.pdf)).

### Parameter Uncertainties

Whether frequentist or Bayesian, the maximum likelihood "point estimate" is only a small part of the picture. What we're really interested in scientifically is the *uncertainty* of the estimates. So simply reporting a point estimate is not appropriate.

In frequentist approaches, "error bars" are generally computed from *Confidence Intervals*, which effectively measure $P(\hat\theta\mid\theta)$, rather than $P(\theta\mid D)$.
It takes some mental gymnastics to relate the confidence interval to the quantity we as scientists have in mind when we say "uncertainty".

In the Bayesian approach, we are actually measuring $P(\theta\mid D)$ from the beginning.

For some approachable reading on frequentist vs. Bayesian uncertainties, I'd suggest [The Fallacy of Placing Confidence in Confidence Intervals](http://learnbayes.org/papers/confidenceIntervalsFallacy/), as well as Jake's (rather opinionated) blog post on the topic, [Confidence, Credibility, and why Frequentism and Science do not Mix](http://jakevdp.github.io/blog/2014/06/12/frequentism-and-bayesianism-3-confidence-credibility/).

### Aside - the denominator in Bayes' rule


The denominator in Bayes' rule is the Evidence $P(D)$

# $$ P(H|D) = \frac{P(D|H) \cdot P(H)}{P(D)} $$

Consider that the evidence can be expressed as an integral using the identities we covered above:
$$
P(D) = \int P(D\mid\theta) P(\theta) d\theta
$$

In other words, it is the integral over the likelihood for *all possible values of theta*.

This means we could have called the Fully Marginalized Likelihood (FML) instead of the evidence. 

When your likelihood is a complicated function of many parameters, computing this integral can become extremely costly (a manifestation of the *curse of dimensionality*), which makes the acronym doubly appropriate in any situation where you actually need it.


In general, for **model fitting**, you can ignore the FML as a simple normalization term. In **model selection**, the FML can become important.

# Moving beyond this controversy requires at least one thing:

If you have a "flat" or uniformative prior:

$$ P(H) = 1$$

Then:

$$ P(H|D) \propto P(D|H) $$


But you already know that $P(D|H)$ is just the likelihood.

## The liklihood function then has to play a dual role:
    
Given a fixed set of model parameters ($\theta = \theta_0$) it is evaluating how likely the data you observed is.


But, you can also consider the likelihood to be a function of the model parameters, given fixed data - we did this last week, deriving the MLE estimate for a homoscedastic Gaussian $N(\mu,\sigma)$.

You cast the likelihood as a function you evaluated at a range of model parameter values.

You'll also have to do this on your homework, except now you've got more than 1 parameter.

### Confidence Interval vs. Credible Region

Even with a flat uniform prior, the two approaches **are not the same.**

In the **frequentist paradigm**, the meaning of the *confidence interval* $\mu_0 \pm \sigma_{\mu}$ is 
the interval that would contain the true $\mu$ (from which the data were drawn) in 68% cases
of a large number of imaginary repeated experiments (each with a different N values of $\{x_i\}$). 

The same interval follows from the **Bayesian approach** with uniform priors.
However, the meaning of that so-called *credible region* is *fundamentally different*: it is the interval
that contains the true $\mu$ with a probability of 68%, given the given dataset (our dear one and only 
dataset - there are no imaginary experiments in Bayesian paradigm). 

This credible region is the relevant quantity in the context of scientific measurements. 

### The Essence of the Bayesian Method 

The basic premise of the Bayesian method is that probability statements are not limited to data, 
but can be made for model parameters and models themselves. Inferences are made by producing 
probability density functions (pdfs); most notably, **model parameters are treated as random variables**.

# In class exercise:

Seeing the drawback of an MLE approach, when you fail to incorporate prior information is easy.

Let's do it with some example data:

In [2]:
### Let's draw a homoscedastic sample of {x_i} from a Gaussian and see what happens with L
# first generate a sample of N points drawn from N(mu,sigma):
%matplotlib inline

import scipy.stats as st
import numpy as np

np.random.seed(42)
sampleSize=3
mu = 1.0
sigma = 0.2 
sample = st.norm(mu, sigma).rvs(sampleSize) 
print(sample)

[1.09934283 0.97234714 1.12953771]


In [3]:
# now compute likelihoods for each point using true mu 
muGrid = np.linspace(0,2,1000)
L1 = st.norm(sample[0], sigma).pdf(muGrid) 
L2 = st.norm(sample[1], sigma).pdf(muGrid) 
L3 = st.norm(sample[2], sigma).pdf(muGrid) 

# what's the total likelihood and where is the maximum likelihood


# plot each of L1, L2, L3 and the total likelihood as a function of mu

In [4]:
# now what happens if I tell you that mu>0.9?
# adjust L1, L2, L3 to reflect this and recompute the total likelihood and maximum likelihood